# Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import statsmodels.api as sm
from statsmodels.api import OLS

In [4]:
lc_census_merged = pd.read_csv('../data/2015_loan_data/pct_accepted_by_zip3_2015_ALL.csv')

In [5]:
lc_census_merged.head()

,index,zip_code,num_accepted,num_rejected,zip_3,Asian,Bachelors_Degree_pct,Black,Families,Family_Households,...,Single_parent_families,Single_parent_no_child_under_18,Some_College_pct,Two,Unemployment_Rate_pct,White,labor_over_16_years,nonwhite_pct,pct_accepted,num_loan_apps_considered
0,0,7,2,89.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022472,91.0
1,1,8,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014706,69.0
2,2,10,379,4165.0,10.0,0.027907,0.198043,0.023016,0.244524,0.244524,...,0.065705,0.030656,0.279196,0.014800,0.065783,0.820135,0.541821,0.179865,0.090996,4544.0
3,3,11,115,1930.0,11.0,0.023936,0.128167,0.171680,0.235750,0.235750,...,0.117448,0.047601,0.282333,0.023795,0.122583,0.382322,0.457280,0.617678,0.059585,2045.0
4,4,12,93,1196.0,12.0,0.015301,0.203040,0.025012,0.255900,0.255900,...,0.071929,0.033828,0.254120,0.020604,0.060160,0.893963,0.536052,0.106037,0.077759,1289.0
